In [18]:
import queries

print(dir(queries))

['SQL_CHART_DATA_BY_SWIMMER_EVENT_SESSION', 'SQL_SESSIONS', 'SQL_SWIMMERS_BY_SESSION', 'SQL_SWIMMERS_EVENTS_BY_SESSION', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__']


In [19]:
statements = []
for q in dir(queries):
    if not q.startswith("__"):
        statements.append(q)
statements

['SQL_CHART_DATA_BY_SWIMMER_EVENT_SESSION',
 'SQL_SESSIONS',
 'SQL_SWIMMERS_BY_SESSION',
 'SQL_SWIMMERS_EVENTS_BY_SESSION']

In [20]:
[sql for sql in dir(queries) if not sql.startswith("__")]

['SQL_CHART_DATA_BY_SWIMMER_EVENT_SESSION',
 'SQL_SESSIONS',
 'SQL_SWIMMERS_BY_SESSION',
 'SQL_SWIMMERS_EVENTS_BY_SESSION']

In [21]:
[x for x in dir(list) if not x.startswith("__")]

['append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [22]:
print([x for x in dir(list) if not x.startswith("__")])

['append', 'clear', 'copy', 'count', 'extend', 'index', 'insert', 'pop', 'remove', 'reverse', 'sort']


In [1]:
import DBcm
db_details = "CoachDB.sqlite3" 


In [24]:
from queries import SQL_SESSIONS

print(SQL_SESSIONS)

 
    select distinct ts from times



In [35]:
with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL_SESSIONS)
    result = db.fetchall()
result

[('2023-11-01 19:07:56',), ('2023-11-13 19:14:56',)]

In [26]:

with DBcm.UseDatabase(db_details) as db:
    db.execute("pragma table_list")
    results = db.fetchall()
results

[('main', 'times', 'table', 4, 0, 0),
 ('main', 'events', 'table', 3, 0, 0),
 ('main', 'sqlite_sequence', 'table', 2, 0, 0),
 ('main', 'swimmers', 'table', 3, 0, 0),
 ('main', 'sqlite_schema', 'table', 5, 0, 0),
 ('temp', 'sqlite_temp_schema', 'table', 5, 0, 0)]

In [27]:
!python update_tables.py

In [29]:
from queries import SQL_SWIMMERS_BY_SESSION
print(SQL_SWIMMERS_BY_SESSION)

 
    select distinct swimmers.name, swimmers.age
    from times, swimmers
    where date(times.ts) = ? and
    times.swimmer_id = swimmers.id
    order by name



In [36]:
with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL_SWIMMERS_BY_SESSION, ("2023-11-13",))
    results = db.fetchall()
results

[('Abi', 10),
 ('Blake', 15),
 ('Darius', 13),
 ('Darius', 8),
 ('Dave', 17),
 ('Katie', 9),
 ('Maria', 9),
 ('Owen', 15)]

In [2]:
from queries import SQL_SWIMMERS_EVENTS_BY_SESSION
print(SQL_SWIMMERS_EVENTS_BY_SESSION)

 
    select distinct events.distance, events.stroke
    from swimmers, events, times
    where times.swimmer_id = swimmers.id and
    times.event_id = events.id and
    (swimmers.name = ? and swimmers.age = ?) and
    date(times.ts) = ?



In [3]:
with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL_SWIMMERS_EVENTS_BY_SESSION, ("Darius", 13, "2023-11-01"))
    results = db.fetchall()
results

[('100m', 'Back'), ('100m', 'Breast'), ('100m', 'Fly'), ('200m', 'IM')]

In [5]:
with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL_SWIMMERS_EVENTS_BY_SESSION, ("Darius", 8, "2023-11-13"))
    results = db.fetchall()
results

[('50m', 'Back'), ('50m', 'Breast'), ('50m', 'Free')]

In [6]:
from queries import SQL_CHART_DATA_BY_SWIMMER_EVENT_SESSION
print(SQL_CHART_DATA_BY_SWIMMER_EVENT_SESSION)

 
    select times.time
    from swimmers, events, times
    where (swimmers.name = ? and swimmers.age = ? ) and
    (events.distance = ? and events.stroke = ? ) and
    times.swimmer_id = swimmers.id and
    times.event_id = events.id and
    date(times.ts) = ?



In [7]:
with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL_CHART_DATA_BY_SWIMMER_EVENT_SESSION,
               ("Darius",13, "100m", "Fly","2023-11-13"))
    results = db.fetchall()
results

[('1:23.22',),
 ('1:21.07',),
 ('1:27.95',),
 ('1:27.95',),
 ('1:28.30',),
 ('1:30.96',)]